#Conectar con la API de Spotify

In [1]:
import requests 
import json
import networkx as nx
import matplotlib.pyplot as plt 
import time
from tqdm import tqdm
import numpy as np
import musicbrainzngs as mb
from os import environ
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from typing import List, Dict

In [2]:
CLIENT_ID = '14c49fbcea5a4439bfa94aa591089353'
CLIENT_SECRET = '710e61b1ca974449a6c7a23c50146cca'
# POST donde le pasamos las clave de la app
response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET})
# Guardamos el bearer token para usarlo en las peticiones de la API
access_token = response.json()['access_token']
headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}

# OAuth_Client_ID = 'SObZ1oUfqqaJS5TLsZ2Ohy4LdqMa3Mpr'
# OAuth_Client_Secret = 'kUXJwU5WqBa5C7CkjCUITfdL6NQMd-4m'
#Identificación

mb.set_useragent('redes_complejas', 0.1) 

In [3]:
CLIENT_ID = '3a283f4bd64241238d019491e4aedca3'
CLIENT_SECRET = 'aee8c6c05d0a42c487c0069b7bd075a8'

response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET})
access_token = response.json()['access_token']
headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}

# OAuth_Client_ID = 'SObZ1oUfqqaJS5TLsZ2Ohy4LdqMa3Mpr'
# OAuth_Client_Secret = 'kUXJwU5WqBa5C7CkjCUITfdL6NQMd-4m'
#Identificación
mb.set_useragent('redes_complejas', 0.1) 

In [4]:
mb.set_useragent('redes_complejas', 0.1) 

# Red de Colaboración de artistas

Idea: agarro un artista random, itero sobre sus canciones, y por cada canción me guardo el nombre de los artistas con los que colaboro. Por cada artista con el que colaboro repito proceso. 

Para encontrar el id de un artista tienen que abrir spotfiy y buscar al artista en cuestion y copiar todos los numeros raros que aparecen al final del link
Osea lo que va despues de : 'https://open.spotify.com/artist/..'

ahora que me guarde todas las canciones, por cada una miro si colaboro con alguien

Creo la red

#Lo hago para varios artistas a manopla, abajo lo automatizo

A veces no reconoce "items" y "artists" pero ni idea porq,. lo corro de nuevo y funca. Probando creo que es porque la api acepta una cierta cantidad de request en un tiempo.

Se me ocurrieron varias cosas del tp.

Para generar la red lo que podríamos hacer es por ejemplo, empezar con un artista cualquiera, chequear sus vecinos y a ellos hacerles todo el proceso de analizar los álbumes y generar los enlaces, así hasta que termine la red o hasta que queramos. Tipo una red ego. Lo que habría que chequear es si se puede pedir el ID de un artista pasándole su nombre.

Sino habría que scrapear una banda de artistas con sus ids y meterlos en un txt. Aunque eso lo terminaremos haciendo seguro.

La otra que se me ocurrió con machine learning es que en vez de estudiar a los años la evolución que quizá tenemos poca data es, pasarle una red incompleta y después pasarle nuevos nodos/enlaces y que nos diga de que género, género musical y país son los nodos nuevos, a ver si puede predecirlos.

La otra manera que se me ocurrió de hacer esto sin machíne learning pero no sé que tan viable es, sería analizar la homofilia de los distintos parámetros de la red y que dándole nuevos nodos, hagamos un código calculé los atributos que minimizan la diferencia de la homofilia original y la de la red, con algún algoritmo a lo modularidad.

#Definimos las cosas en funciones para agilizar el análisis.

In [12]:
def conseguir_id_y_genero(artista):
    """
    Recibe
        artista: Nombre del artista del cual se quiere su ID
    Devuelve:
        id: La ID del artista y el/los generos del artista
    """
    # End point para obtener los audio features. Esto se saca de la referencia de la documentación
    url = 'https://api.spotify.com/v1/search'

    # Búsqueda. Acá sí hay que pasarselo como parámetros
    params = {'q': f"{artista}", 'type': 'artist', 'limit': '5'}

    # En este caso no lleva ningún parámetro, el id de la canción va directamente en el url
    response = requests.get(url, params = params, headers = headers)

    # Vemos el json de la respuesta
    json_data = response.json()
    genero = json_data['artists']['items'][0]['genres']
    
    # Con la siguiente lista conseguimos el ID del primer resultado de la busqueda (creo)
    id = json_data["artists"]["items"][0]["id"]

    return (id,genero)

In [13]:

def conseguir_cuadrivector(artista): 
    """
    Recibe
        nombre: artista 
    Devuelve:
        lista de cuadrivectores: [(artista,artistas_colaborador1,idcancion,fecha), (devueltaigual),..., ]
    """
    
    id = conseguir_id_y_genero(artista)[0]
    canciones = []
    #pido la data de los albumes del artista
    url = f'https://api.spotify.com/v1/artists/{id}/albums'
    response = requests.get(url, params = {}, headers = headers)
    json_data = response.json()
    items = json_data['items']
    #me guardo todos sus albumes
    
    

    canciones_junto_a_id = []
    
    
    for indice_album,item in enumerate(items):
        año_album = items[indice_album]['release_date']
        nombre_album = items[indice_album]['id']
        
        url = f'https://api.spotify.com/v1/albums/{nombre_album}/tracks'
        response = requests.get(url, params = {}, headers = headers)
        data_albumes = response.json()
        
        #itero sobre todas las canciones de cada album y la guardo en la lista de canciones
        data_canciones = data_albumes['items']
        
        for index_cancion in range(len(data_canciones)):
            
            song = data_canciones[index_cancion]['id']
            
            url = f' https://api.spotify.com/v1/tracks/{song}'
            response = requests.get(url, params = {}, headers = headers)
            json_data = response.json()
            
            for index_colaborador in range(len(json_data['artists'])):
                
                colaborador = json_data['artists'][index_colaborador]['name'] 

                canciones_junto_a_id.append((artista,colaborador,song,año_album))
                if canciones_junto_a_id[-1][0] == canciones_junto_a_id[-1][1]: #   AGREGAMOS ESTO PARA quitar autoenlaces
                    canciones_junto_a_id.pop(-1)
    
    
    return canciones_junto_a_id

In [14]:
def crear_enlaces(nombre_artista,colaboraciones):
    # Crea una lista con todos los enlaces
    enlaces = [(nombre_artista,colaboracion) for colaboracion in colaboraciones]

    return enlaces

In [15]:
def conseguir_pais_y_genero(nombre_artista): #AGREGAR ACA SI QUEREMOS ALGUN OTRO ATRIBUTO
    
    dic_atributos = {}
    
    data = mb.search_artists(artist = nombre_artista, limit= 1)

    if data["artist-count"] != 0: #SI ENCUENTRA EL ARTISTA EN MUSIC BRANDZ
        if "country" in data["artist-list"][0]:
            pais = data["artist-list"][0]["country"]
            dic_atributos["pais"] = pais
            
        if "gender" in data["artist-list"][0]:
            genero = data["artist-list"][0]["gender"]
            dic_atributos["genero"] = genero
    
    return dic_atributos

In [16]:

# https://github.com/plamere/spotipy/blob/master/examples/playlist_tracks.py


# Authenticate to Spotify
def authenticate(cliend_id: str, client_secret: str) -> spotipy.client.Spotify:
    sp = spotipy.Spotify(
        client_credentials_manager=SpotifyClientCredentials(
            client_id=cliend_id,
            client_secret=client_secret
        )
    )

    return sp

autentificador = authenticate('14c49fbcea5a4439bfa94aa591089353','710e61b1ca974449a6c7a23c50146cca')
# Number of tracks available in the playlist
def get_pl_length(sp: spotipy.client.Spotify, pl_uri: str) -> int:
    return sp.playlist_tracks(
        pl_uri,
        offset=0,
        fields="total"
    )["total"]
    
    
def get_tracks_artist_info(sp: spotipy.client.Spotify, pl_uri: str) -> List[List[Dict]]:
    artists_info = list()
    # Start retrieving tracks from the beginning of the playlist
    offset = 0
    pl_length = get_pl_length(sp, pl_uri)

    # Playlist track retrieval only fetches 100 tracks at a time, hence\
    # the loop to keep retrieving until we reach the end of the playlist
    while offset != pl_length:
        # Get the next batch of tracks
        pl_tracks = sp.playlist_tracks(
            pl_uri,
            offset=offset,
            fields="items.track"
        )

        # Get the list with the info about the artists of each track from the\
        # latest batch and append it to the running list
        
        [artists_info.append(pl_item["track"]["artists"]) for pl_item in pl_tracks["items"]]

        # Update the offset
        offset += len(pl_tracks["items"])

    return artists_info



In [17]:
artistas_argentinos = []

data_artistas = get_tracks_artist_info(autentificador,"https://open.spotify.com/playlist/6oF4i9FMoWGfvwUUI7IpgH?si=604e37fad5304c52")
for artista in data_artistas:
    nombre_artista = artista[0]["name"]
    pais_y_genero_artista = conseguir_pais_y_genero(nombre_artista)
    try:
        if pais_y_genero_artista["pais"] == "AR" and nombre_artista not in artistas_argentinos:
            artistas_argentinos.append(nombre_artista)
    except:
        print()

KeyboardInterrupt: 

In [10]:
print(artistas_argentinos)
print(len(artistas_argentinos)) ##LA PLAYLIST TIENE 50 ARTISTAS, SOLO 3 NO TIENEN "COUNTRY" o no son AR.

['Charly García', 'Soda Stereo', 'Los Abuelos De La Nada', 'Los Enanitos Verdes', 'Virus', 'WOS', 'David Lebón', 'Los Piojos', 'Divididos', 'Gustavo Cerati', 'Conociendo Rusia', 'Fito Paez', 'Las Pastillas del Abuelo', 'Estelares', 'Vicentico', 'Luis Alberto Spinetta', 'Indio Solari', 'Sumo', 'Eruca Sativa', 'Emmanuel Horvilleur', 'Dante Spinetta', 'Skay Beilinson', 'Callejeros', 'Attaque 77', 'La Franela', 'Intoxicados', 'Los Caballeros De La Quema', 'Los Fabulosos Cadillacs', 'Tan Bionica', 'Bersuit Vergarabat', 'Las Pelotas', 'La Beriso', 'La Renga', 'Turf', 'Andrés Calamaro', 'Los Auténticos Decadentes', 'Gustavo Cordera', 'Serú Girán', 'Pappo', 'Dread Mar I', 'Babasónicos', 'Ivan Noble', 'Guasones']
43


In [11]:
#URLS DE PLAYLIST CON ARTISTAS ARGENTINOS:
#rock nacional, folcrore argentino, los 90 argentina.

artistas_argentinos = []
artistas_quiza_argentinos = []

playlists = ["https://open.spotify.com/playlist/37i9dQZF1DX2v4h8QERQPb?si=f0244631d0084cb6","https://open.spotify.com/playlist/4BfzTfnImLBKkhYZwgsRMO?si=562120a745954c53","https://open.spotify.com/playlist/6GxBZn0yu0miclv2bhImBd?si=c513cd4ed3784604", "https://open.spotify.com/playlist/37i9dQZF1DX8KUmmjlkOod?si=fbfbb389e3034a54","https://open.spotify.com/playlist/6oF4i9FMoWGfvwUUI7IpgH?si=604e37fad5304c52"] 

for playlist in playlists:
    data_artistas = get_tracks_artist_info(autentificador,playlist)
    
    
    for artista in data_artistas:
        
        nombre_artista = artista[0]["name"]
        # print(nombre_artista)
        
        pais_y_genero_artista = conseguir_pais_y_genero(nombre_artista)
        
        try:
            if pais_y_genero_artista["pais"] == "AR" and nombre_artista not in artistas_argentinos:
                artistas_argentinos.append(nombre_artista)
        except:
            if nombre_artista not in artistas_quiza_argentinos:
            
                artistas_quiza_argentinos.append(nombre_artista)

KeyboardInterrupt: 

La siguiente celda es para mergear la que tira last fm con la de Nico

In [31]:
import requests
from bs4 import BeautifulSoup as BS 
import time
import numpy as np
response = requests.get("https://www.last.fm/es/tag/argentina/artists")

# Vemos el contenido que nos devolvió
print(response.status_code)
soup = BS(response.content)
urls = []
for i in range(2,49):
    urls.append(f'https://www.last.fm/es/tag/argentina/artists?page={i}')
artistas = []
artistas_argentinos = []
artistas_quiza_argentinos = []
for url in urls:
    response = requests.get(url)
    soup = BS(response.content)
    elements = soup.findAll(attrs = {"class": "big-artist-list-title"})
    for element in elements:
        artistas.append(element.a.text)

for artista in artistas:  
        pais_y_genero_artista = conseguir_pais_y_genero(artista)
        time.sleep(1)
        try:
            if pais_y_genero_artista["pais"] == "AR" and nombre_artista not in artistas_argentinos:
                artistas_argentinos.append(nombre_artista)
        except:
            if artista not in artistas_quiza_argentinos:
                artistas_quiza_argentinos.append(nombre_artista)
artistas_argentinos_nico = ['Bardero$', 'WOS', 'YSY A', 'Nicki Nicole', 'Dani Ribba', 'Dillom', 'Bizarrap', 'Trueno', 'Homer El Mero Mero', 'ACRU', 'Franky Style', 'Kris Alaniz', 'Kodigo', 'Fianru', 'Lil Troca', 'Veeyam', 'Marcianos Crew', 'KHEA', 'Frijo', 'FMK', 'Lucho SSJ', 'Pekeño 77', 'Chaqueño Palavecino', 'Soledad', 'Horacio Guarany', 'Los Huayra', 'Facundo Toro', 'Los Nombradores del Alba', 'Nacho Prado Y Daniel Campos', 'Sandra Luna', 'Los Chalchaleros', 'Jorge Rojas', 'Daniel Toro', 'Los Nocheros', 'Luciano Pereyra', 'Roberto Carabajal', 'Abel Pintos', 'Peteco Carabajal', 'Roxana Carabajal', 'Mercedes Sosa', 'Raly Barrionuevo', 'Andrés Calamaro', 'Soda Stereo', 'Los Fabulosos Cadillacs', 'Los Auténticos Decadentes', 'Los Enanitos Verdes', 'Luis Alberto Spinetta', 'Gustavo Cerati', 'Fito Paez', 'Los Pericos', 'Patricio Rey y sus Redonditos de Ricota', 'Vilma Palma e Vampiros', 'Los Caballeros De La Quema', 'Attaque 77', 'Diego Torres', 'Gilda', 'Grupo Sombras', 'El Soldado', 'Fabiana Cantilo', 'Los Piojos', 'Bersuit Vergarabat', 'La Portuaria', 'Los Ladrones Sueltos', 'Memphis La Blusera', 'La Mosca Tse-Tse', 'Divididos', 'Sergio Denis', 'León Gieco', 'Charly García', 'Rafaga', 'Illya Kuryaki & The Valderramas', 'La Zimbabwe', 'Rata Blanca', 'Manuel Wirzt', 'Alcides', 'Los Abuelos De La Nada', 'Virus', 'David Lebón', 'Conociendo Rusia', 'Las Pastillas del Abuelo', 'Estelares', 'Vicentico', 'Indio Solari', 'Sumo', 'Eruca Sativa', 'Emmanuel Horvilleur', 'Dante Spinetta', 'Skay Beilinson', 'Callejeros', 'La Franela', 'Intoxicados', 'Tan Bionica', 'Las Pelotas', 'La Beriso', 'La Renga', 'Turf', 'Gustavo Cordera', 'Serú Girán', 'Pappo', 'Dread Mar I', 'Babasónicos', 'Ivan Noble', 'Guasones']
artistas_quiza_argentinos_nico = ['Duki', 'El Triangulo Estudio', 'Ronpe 99', 'G Sony', 'Slim Dee', 'Ronpe 99', 'Flaco Vazquez', 'Slim Dee', 'Brapis', 'Wolty', 'Dtoke', 'Iacho', 'Rapstinencia', 'La Joaqui', 'Urbanse', 'Urbanse', 'Wolty', 'Zica', 'Ronpe 99', 'Zica', 'Ronpe 99', 'Duki', 'Duki', 'Khea;Omar Varela', 'LIT killah', 'Duki', 'Duki', 'Duki', 'LIT killah', 'Duki', 'Neo Pistea', 'Seven Kayne', 'Duki', 'LIT killah', 'Duki', 'Duki', 'Duki', 'Midel', 'Seven Kayne', 'Ecko', 'Ecko', 'Luck Ra;Seven Kayne', 'Cumbia Trampa', 'Duki', 'Duki', 'Zanto', 'Neo Pistea', 'Felp 22', 'LIT killah', 'Seven Kayne', 'Neo Pistea', 'Duki', 'Duki', 'Duki', 'Duki', 'Duki', 'Duki', 'Antezanax 247', 'Campedrinos', 'Sergio Galleguillo', 'Tamara Castro', 'Los Rojas', 'Campedrinos', 'Campedrinos', 'Campedrinos', 'Campedrinos', 'Campedrinos', 'Daniela Toro', 'Sergio Galleguillo Y Los Amigos', 'Campedrinos', 'Los Carabajal', 'Tamara Castro', 'Campedrinos', 'Tamara Castro', 'Sergio Galleguillo', 'Sergio Galleguillo Y Los Amigos', 'Amboé', 'Campedrinos', 'Campedrinos', 'Amar Azul']

#mergeo lo de nico con lo de ari

artistas_argentinos = artistas_argentinos+artistas_argentinos_nico
artistas_quiza_argentinos = artistas_quiza_argentinos + artistas_quiza_argentinos_nico

200


In [ ]:
artistas_argentinos = ['Los Abuelos De La Nada', 'Bardero$', 'WOS', 'YSY A', 'Nicki Nicole', 'Dani Ribba', 'Dillom', 'Bizarrap', 'Trueno', 'Homer El Mero Mero', 'ACRU', 'Franky Style', 'Kris Alaniz', 'Kodigo', 'Fianru', 'Lil Troca', 'Veeyam', 'Marcianos Crew', 'KHEA', 'Frijo', 'FMK', 'Lucho SSJ', 'Pekeño 77', 'Chaqueño Palavecino', 'Soledad', 'Horacio Guarany', 'Los Huayra', 'Facundo Toro', 'Los Nombradores del Alba', 'Nacho Prado Y Daniel Campos', 'Sandra Luna', 'Los Chalchaleros', 'Jorge Rojas', 'Daniel Toro', 'Los Nocheros', 'Luciano Pereyra', 'Roberto Carabajal', 'Abel Pintos', 'Peteco Carabajal', 'Roxana Carabajal', 'Mercedes Sosa', 'Raly Barrionuevo', 'Andrés Calamaro', 'Soda Stereo', 'Los Fabulosos Cadillacs', 'Los Auténticos Decadentes', 'Los Enanitos Verdes', 'Luis Alberto Spinetta', 'Gustavo Cerati', 'Fito Paez', 'Los Pericos', 'Patricio Rey y sus Redonditos de Ricota', 'Vilma Palma e Vampiros', 'Los Caballeros De La Quema', 'Attaque 77', 'Diego Torres', 'Gilda', 'Grupo Sombras', 'El Soldado', 'Fabiana Cantilo', 'Los Piojos', 'Bersuit Vergarabat', 'La Portuaria', 'Los Ladrones Sueltos', 'Memphis La Blusera', 'La Mosca Tse-Tse', 'Divididos', 'Sergio Denis', 'León Gieco', 'Charly García', 'Rafaga', 'Illya Kuryaki & The Valderramas', 'La Zimbabwe', 'Rata Blanca', 'Manuel Wirzt', 'Alcides', 'Los Abuelos De La Nada', 'Virus', 'David Lebón', 'Conociendo Rusia', 'Las Pastillas del Abuelo', 'Estelares', 'Vicentico', 'Indio Solari', 'Sumo', 'Eruca Sativa', 'Emmanuel Horvilleur', 'Dante Spinetta', 'Skay Beilinson', 'Callejeros', 'La Franela', 'Intoxicados', 'Tan Bionica', 'Las Pelotas', 'La Beriso', 'La Renga', 'Turf', 'Gustavo Cordera', 'Serú Girán', 'Pappo', 'Dread Mar I', 'Babasónicos', 'Ivan Noble', 'Guasones']

In [34]:
print(artistas_argentinos)
print(artistas_quiza_argentinos)

['Los Abuelos De La Nada', 'Bardero$', 'WOS', 'YSY A', 'Nicki Nicole', 'Dani Ribba', 'Dillom', 'Bizarrap', 'Trueno', 'Homer El Mero Mero', 'ACRU', 'Franky Style', 'Kris Alaniz', 'Kodigo', 'Fianru', 'Lil Troca', 'Veeyam', 'Marcianos Crew', 'KHEA', 'Frijo', 'FMK', 'Lucho SSJ', 'Pekeño 77', 'Chaqueño Palavecino', 'Soledad', 'Horacio Guarany', 'Los Huayra', 'Facundo Toro', 'Los Nombradores del Alba', 'Nacho Prado Y Daniel Campos', 'Sandra Luna', 'Los Chalchaleros', 'Jorge Rojas', 'Daniel Toro', 'Los Nocheros', 'Luciano Pereyra', 'Roberto Carabajal', 'Abel Pintos', 'Peteco Carabajal', 'Roxana Carabajal', 'Mercedes Sosa', 'Raly Barrionuevo', 'Andrés Calamaro', 'Soda Stereo', 'Los Fabulosos Cadillacs', 'Los Auténticos Decadentes', 'Los Enanitos Verdes', 'Luis Alberto Spinetta', 'Gustavo Cerati', 'Fito Paez', 'Los Pericos', 'Patricio Rey y sus Redonditos de Ricota', 'Vilma Palma e Vampiros', 'Los Caballeros De La Quema', 'Attaque 77', 'Diego Torres', 'Gilda', 'Grupo Sombras', 'El Soldado', 'Fa

In [33]:
# Defino una lista con todos los artistas

# Defino una lista para guardar los enlaces
cuadrivectores_artistas_argentinos = []

for artista in artistas_argentinos:
    lista_cuadrivectores = conseguir_cuadrivector(artista)
    for vector in lista_cuadrivectores:
        nombre_colaborador = vector[1]
        pais_y_genero_colaborador = conseguir_pais_y_genero(nombre_colaborador)
        time.sleep(1.5)
        try:
            if pais_y_genero_colaborador["pais"] == "AR":
                cuadrivectores_artistas_argentinos.append(vector)
        except:
            pass

KeyError: 'artists'

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
c:\Users\ariro\OneDrive\Desktop\Redes\Redes-complejas\codigo nico.ipynb Celda 23 in <cell line: 6>()
      4 cuadrivectores_artistas_argentinos = []
      6 for artista in artistas_argentinos:
----> 7     lista_cuadrivectores = conseguir_cuadrivector(artista)
      8     for vector in lista_cuadrivectores:
      9         nombre_colaborador = vector[1]

c:\Users\ariro\OneDrive\Desktop\Redes\Redes-complejas\codigo nico.ipynb Celda 23 in conseguir_cuadrivector(artista)
      1 def conseguir_cuadrivector(artista): 
      2     """
      3     Recibe
      4         nombre: artista 
      5     Devuelve:
      6         lista de cuadrivectores: [(artista,artistas_colaborador1,idcancion,fecha), (devueltaigual),..., ]
      7     """
----> 9     id = conseguir_id_y_genero(artista)[0]
     10     canciones = []
     11     #pido la data de los albumes del artista

c:\Users\ariro\OneDrive\Desktop\Redes\Redes-complejas\codigo nico.ipynb Celda 23 in conseguir_id_y_genero(artista)
     17 # Vemos el json de la respuesta
     18 json_data = response.json()
---> 19 genero = json_data['artists']['items'][0]['genres']
     21 # Con la siguiente lista conseguimos el ID del primer resultado de la busqueda (creo)
     22 id = json_data["artists"]["items"][0]["id"]

KeyError: 'artists'

In [6]:

cuadrivectores_artistas_argentinos = [('Bardero$', 'Franky Style', '1remv0DkW7pPBHsOGtyON1', '2020-10-23'), ('Bardero$', 'Chulu', '1remv0DkW7pPBHsOGtyON1', '2020-10-23'), ('Bardero$', 'Franky Style', '3kJkC0W2QMs8acfS4ssB6f', '2020-10-23'), ('Bardero$', 'Chulu', '2UhrNOPfjhpqH4v0fMYLIQ', '2020-10-23'), ('Bardero$', 'Franky Style', '6keZVGv8A4cX5Q4NfIV61t', '2020-10-23'), ('Bardero$', 'Homer El Mero Mero', '4ZPHivCYG5nqs7UWwXbgK2', '2020-10-23'), ('Bardero$', 'Franky Style', '4ZPHivCYG5nqs7UWwXbgK2', '2020-10-23'), ('Bardero$', 'Homer El Mero Mero', '5B4EkBBQlZJsS1qWPjmq0B', '2020-10-23'), ('Bardero$', 'Lil Troca', '5B4EkBBQlZJsS1qWPjmq0B', '2020-10-23'), ('Bardero$', 'Franky Style', '4ATtunTXSzWkqUTqOZjZts', '2020-10-23'), ('Bardero$', 'Lil Troca', '4ATtunTXSzWkqUTqOZjZts', '2020-10-23'), ('Bardero$', 'Franky Style', '7JyowjMQnogOnpVk0HCKfG', '2020-10-23'), ('Bardero$', 'Franky Style', '2e11sHNDJS7cYDLgOiVz6X', '2020-10-23'), ('Bardero$', 'Chulu', '2e11sHNDJS7cYDLgOiVz6X', '2020-10-23'), ('Bardero$', 'Franky Style', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Bardero$', 'Homer El Mero Mero', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Bardero$', 'Lil Troca', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Bardero$', 'Chulu', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Bardero$', 'Homer El Mero Mero', '3MtToQhywGW6Q98cYcHYN1', '2020-07-07'), ('Bardero$', 'Homer El Mero Mero', '1CzN58609yKsgo4Z9ba4cl', '2020-07-07'), ('Bardero$', 'Homer El Mero Mero', '0Kros3xlCtmyHTarar1SZ3', '2022-10-18'), ('Bardero$', 'Lil Troca', '0frbDonZlnjEaxKnPKclb3', '2022-02-08'), ('Bardero$', 'Franky Style', '0frbDonZlnjEaxKnPKclb3', '2022-02-08'), ('Bardero$', 'Chulu', '0frbDonZlnjEaxKnPKclb3', '2022-02-08'), ('Bardero$', 'Chulu', '6Ujwe7jHe1vv8b6uUbsYbQ', '2020-08-14'), ('Bardero$', 'Franky Style', '6Ujwe7jHe1vv8b6uUbsYbQ', '2020-08-14'), ('Bardero$', 'Homer El Mero Mero', '7BCMcCVPytXI4GHEz3dQmF', '2020-05-27'), ('Bardero$', 'Homer El Mero Mero', '1EmehmbURkCcxpHcxcXgDP', '2020-04-07'), ('Bardero$', 'Homer El Mero Mero', '5nRy9fFwzFMyMxdYOSLHqo', '2020-02-18'), ('WOS', 'Ricardo Mollo', '0BTJ7QpDRjinzAy8KQMAp8', '2021-11-18'), ('WOS', 'CA7RIEL', '0Wz6TToEPwxzH482fuGEwO', '2021-11-18'), ('WOS', 'Nicki Nicole', '2y2iEADc8O0Vv5IKHSs21i', '2021-11-18'), ('WOS', 'LOUTA', '0bhWW1Bve1Ng6kEm8WjiQA', '2022-06-16'), ('WOS', 'La Bomba de Tiempo', '2MuDW3QK1SAk35OnTnDLLp', '2019-10-25'), ('WOS', 'Baltasar Comotto', '2MuDW3QK1SAk35OnTnDLLp', '2019-10-25'), ('WOS', 'ACRU', '06b0Au6pvZiO03eKTCvgWh', '2019-07-19'), ('YSY A', 'CLUB HATS', '2HB69IWEfPkIMyBB7umsSf', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '2HB69IWEfPkIMyBB7umsSf', '2021-11-11'), ('YSY A', 'CLUB HATS', '28wLks1lbwQ72wZNmQ1TLS', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '28wLks1lbwQ72wZNmQ1TLS', '2021-11-11'), ('YSY A', 'CLUB HATS', '2YVd5KUxkBnTEtPuqFR5uc', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '2YVd5KUxkBnTEtPuqFR5uc', '2021-11-11'), ('YSY A', 'CLUB HATS', '0Xu2p23OYBluVeY5w3SEH2', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '0Xu2p23OYBluVeY5w3SEH2', '2021-11-11'), ('YSY A', 'CLUB HATS', '2Sq5ptp63NfvmxdkdXFopO', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '2Sq5ptp63NfvmxdkdXFopO', '2021-11-11'), ('YSY A', 'CLUB HATS', '6gIr3qztRUa6TV1jWfjs7V', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '6gIr3qztRUa6TV1jWfjs7V', '2021-11-11'), ('YSY A', 'CLUB HATS', '38uRnUZTf0eCKfKNnVUSaT', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '38uRnUZTf0eCKfKNnVUSaT', '2021-11-11'), ('YSY A', 'CLUB HATS', '7hbodg9YQKWka80HWOxram', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '7hbodg9YQKWka80HWOxram', '2021-11-11'), ('YSY A', 'CLUB HATS', '6jxC7ERCNlDeVbFXHYEUNN', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '6jxC7ERCNlDeVbFXHYEUNN', '2021-11-11'), ('YSY A', 'Sixto Yegros', '6jxC7ERCNlDeVbFXHYEUNN', '2021-11-11'), ('YSY A', 'CLUB HATS', '2JjtlHKKjMpFuCcT1ZO2c8', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '2JjtlHKKjMpFuCcT1ZO2c8', '2021-11-11'), ('YSY A', 'CLUB HATS', '16QIoTktQs6hpRpn6zgjtI', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '16QIoTktQs6hpRpn6zgjtI', '2021-11-11'), ('YSY A', 'CLUB HATS', '7ewHmbeqryP2T7rnbt9wpn', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '7ewHmbeqryP2T7rnbt9wpn', '2021-11-11'), ('YSY A', 'CLUB HATS', '2FQXQQpxrOYY9sr6aSsbt5', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '2FQXQQpxrOYY9sr6aSsbt5', '2021-11-11'), ('YSY A', 'Obie Wanshot', '2FQXQQpxrOYY9sr6aSsbt5', '2021-11-11'), ('YSY A', 'CLUB HATS', '0S3ZQgpuP4QBle0pESON8i', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '0S3ZQgpuP4QBle0pESON8i', '2021-11-11'), ('YSY A', 'CLUB HATS', '0tKAWG4Rt4Q9Vt96rOzXmh', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '0tKAWG4Rt4Q9Vt96rOzXmh', '2021-11-11'), ('YSY A', 'CLUB HATS', '1JszOi9nTXFR5gxFyczhEw', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '1JszOi9nTXFR5gxFyczhEw', '2021-11-11'), ('YSY A', 'CLUB HATS', '394ASxIHaPcSZ4FSzTz4d7', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '394ASxIHaPcSZ4FSzTz4d7', '2021-11-11'), ('YSY A', 'CLUB HATS', '2E5Spb1g73AeISkpH6Rs1d', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '2E5Spb1g73AeISkpH6Rs1d', '2021-11-11'), ('YSY A', 'CLUB HATS', '4ioyBhsSp4X4Ev3Yw0aTga', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '4ioyBhsSp4X4Ev3Yw0aTga', '2021-11-11'), ('YSY A', 'Sixto Yegros', '4ioyBhsSp4X4Ev3Yw0aTga', '2021-11-11'), ('YSY A', 'CLUB HATS', '1IwXilsUGJFd7zpMDQgRun', '2021-11-11'), ('YSY A', 'SPONSOR DIOS', '1IwXilsUGJFd7zpMDQgRun', '2021-11-11'), ('YSY A', 'Obie Wanshot', '4ffqSTFwPUdDLLfLa8OuC6', '2018-11-11'), ('YSY A', 'Marcianos Crew', '2UJxBdSItReWTTQUvUhVLv', '2018-11-11'), ('YSY A', 'CLUB HATS', '1ND2wzmMW3t9HHaHPMIaNo', '2021-10-31'), ('YSY A', 'SPONSOR DIOS', '1ND2wzmMW3t9HHaHPMIaNo', '2021-10-31'), ('YSY A', 'Yesan 雪山', '0GnAvYHtwWi1x2BosdNDf5', '2021-09-05'), ('YSY A', 'SPONSOR DIOS', '0GnAvYHtwWi1x2BosdNDf5', '2021-09-05'), ('YSY A', 'SPONSOR DIOS', '3VTjIwrfPrnsI49lsF1mMs', '2021-08-22'), ('YSY A', 'SPONSOR DIOS', '5VxQlDyKX3gpdOXKTXUngX', '2021-08-08'), ('YSY A', 'Sixto Yegros', '3NsMTx5SWy3z7LRt7cl0NY', '2021-07-12'), ('YSY A', 'CLUB HATS', '3NsMTx5SWy3z7LRt7cl0NY', '2021-07-12'), ('YSY A', 'SPONSOR DIOS', '3NsMTx5SWy3z7LRt7cl0NY', '2021-07-12'), ('YSY A', 'SPONSOR DIOS', '0YN9ifrMXya1HAiGoQfKhf', '2021-06-27'), ('YSY A', 'Asan', '4L9WoTpNvdFv8SPW7koxWC', '2021-05-25'), ('YSY A', 'CLUB HATS', '5u1azLYFRISTVqC6FQWLYd', '2021-04-25'), ('YSY A', 'SPONSOR DIOS', '5u1azLYFRISTVqC6FQWLYd', '2021-04-25'), ('YSY A', 'Bizarrap', '1xXQCSt0xkfeGhe18ey2p8', '2021-01-27'), ('YSY A', 'CLUB HATS', '4ycnpTN1gmqOfxkLfBNbWB', '2020-11-11'), ('YSY A', 'Yesan 雪山', '4ycnpTN1gmqOfxkLfBNbWB', '2020-11-11'), ('YSY A', 'CLUB HATS', '5HlxMGUTMNsnHWJSzdQcxN', '2020-11-11'), ('YSY A', 'Yesan 雪山', '5HlxMGUTMNsnHWJSzdQcxN', '2020-11-11'), ('YSY A', 'CLUB HATS', '0Bue0KeMCaYP2RNn0OTDPq', '2020-11-11'), ('YSY A', 'Yesan 雪山', '0Bue0KeMCaYP2RNn0OTDPq', '2020-11-11'), ('YSY A', 'CLUB HATS', '0jizWh29OJMwTFZZ01nzxp', '2020-11-11'), ('YSY A', 'Yesan 雪山', '0jizWh29OJMwTFZZ01nzxp', '2020-11-11'), ('YSY A', 'CLUB HATS', '5zZwYsMztTGMPwWvlYoCfp', '2020-11-11'), ('YSY A', 'Yesan 雪山', '5zZwYsMztTGMPwWvlYoCfp', '2020-11-11'), ('YSY A', 'CLUB HATS', '6NnqP2DazV1Q1mqmV1JNYC', '2020-11-11'), ('YSY A', 'Yesan 雪山', '6NnqP2DazV1Q1mqmV1JNYC', '2020-11-11'), ('YSY A', 'CLUB HATS', '1mDhaG7lAIduxFDLXJKjm0', '2020-11-09'), ('YSY A', 'Yesan 雪山', '1mDhaG7lAIduxFDLXJKjm0', '2020-11-09'), ('YSY A', 'Bizarrap', '2byu6eeLWaBq88ZtusqU4V', '2020-10-26'), ('YSY A', 'CLUB HATS', '6TI490muJ6RgFsYJc626BZ', '2020-10-18'), ('YSY A', 'Yesan 雪山', '7qf15ZBTnAyOZX5UIWrsKj', '2020-10-11'), ('YSY A', 'Cucuza Castiello', '2kGq3Hvr0714szg3vpzkJa', '2020-10-08'), ('YSY A', 'Amelita Baltar', '2kGq3Hvr0714szg3vpzkJa', '2020-10-08'), ('Nicki Nicole', 'Dread Mar I', '0PgZvzLzdbwJC0a0UZvZle', '2021-10-28'), ('Nicki Nicole', 'Bizarrap', '0PgZvzLzdbwJC0a0UZvZle', '2021-10-28'), ('Nicki Nicole', 'Trueno', '1B7Wxx58QV4xrCjxI4Uu3i', '2021-10-28'), ('Nicki Nicole', 'Bizarrap', '1B7Wxx58QV4xrCjxI4Uu3i', '2021-10-28'), ('Nicki Nicole', 'Tiago PZK', '57V4CXFQ5I5ok4yCDxPEjW', '2021-10-28'), ('Nicki Nicole', 'Cazzu', '7EehFSuTqX5ufeVwajlnNa', '2019-11-08'), ('Nicki Nicole', 'Bizarrap', '4DfhsWuqZonO5URxvwkBSK', '2019-11-08'), ('Nicki Nicole', 'Bizarrap', '42ahuAFlYGjIGBpVdHcLij', '2021-07-28'), ('Dani Ribba', 'Tiago PZK', '73zRxy5QmTGa5C8IAS3axW', '2022-10-20'), ('Dani Ribba', 'Trueno', '12GYDIfcTO5fEbWuU9kgCZ', '2022-10-20'), ('Dani Ribba', 'FMK', '5JH2j55T7jhjhr4ZsC8tZs', '2022-03-10'), ('Dani Ribba', 'Tiago PZK', '6t4YfQwwCZS5NIVJelVa9y', '2021-11-30'), ('Dani Ribba', 'Bizarrap', '4ggrH4V2eKpf2xDfHMuRrq', '2020-03-04'), ('Dani Ribba', 'Bizarrap', '1jKq6jcjPAd2awwuYiPswp', '2019-11-07'), ('Dani Ribba', 'Lucho SSJ', '3WMtB1EbqxQUfnvk5Kaygm', '2019-03-30'), ('Dillom', 'Muerejoven', '6YbzhSDVIz6zG9Hf7CoNUc', '2021-12-01'), ('Dillom', 'Saramalacara', '3z8rcgJdQo2g2G4VijeLpl', '2021-12-01'), ('Dillom', 'Miranda!', '5Eusj6NCSedMJdPMn6V7EG', '2022-08-26'), ('Dillom', 'Broke Carrey', '1wEZvDYEUKO2F83LgjThSe', '2022-04-28'), ('Dillom', 'Punga', '0KlXfxDmQqz4ajLc4cksyy', '2021-01-22'), ('Dillom', 'Muerejoven', '4wwcHxuORt22e6j7BOablC', '2020-05-29'), ('Dillom', 'Parcas', '4wwcHxuORt22e6j7BOablC', '2020-05-29'), ('Dillom', 'Muerejoven', '4pl91a0QtmM1BwZW1Vkdcy', '2020-03-16'), ('Dillom', 'KAKTOV', '4pl91a0QtmM1BwZW1Vkdcy', '2020-03-16'), ('Dillom', 'KAKTOV', '1WRIN4Liz3nmoE5njV8aNO', '2019-12-13'), ('Dillom', 'Muerejoven', '6TesHoNPZGdcVjllhAuKJh', '2019-09-27'), ('Dillom', 'Muerejoven', '4V7aTjULqQX7aaaslStMNW', '2019-07-18'), ('Dillom', 'Ill Quentin', '4SkkG8sOjt9dyBJ8xv0CgD', '2019-07-18'), ('Bizarrap', 'Paulo Londra', '4F4bzCA8RWoyKQDfeFGolM', '2022-04-25'), ('Bizarrap', 'Trueno', '3monFIRNCDt9we37DDvCCK', '2022-02-17'), ('Bizarrap', 'Pablo Lescano', '7nXutPtu8epp2VJ300iN2A', '2022-01-19'), ('Bizarrap', 'Tiago PZK', '6VAS7m73rDopQe7BdrFMpD', '2021-12-29'), ('Bizarrap', 'Nicki Nicole', '42ahuAFlYGjIGBpVdHcLij', '2021-07-28'), ('Trueno', 'Victor Heredia', '73zQG7t3tZDyePEflVSqJJ', '2022-05-12'), ('Trueno', 'Bizarrap', '2jPufm4Nf3JVe0fdLXA67A', '2022-05-12'), ('Trueno', 'Nicki Nicole', '0toFR0lJQZL0rqO12zqKLF', '2021-06-28'), ('Trueno', 'Taiu', '2FPE3JcSvRoCNSC96KAyvB', '2020-07-23'), ('Trueno', 'Yesan 雪山', '2FPE3JcSvRoCNSC96KAyvB', '2020-07-23'), ('Trueno', 'Taiu', '16tcwFXBCSmSxLj5vnlsU9', '2020-07-23'), ('Trueno', 'Taiu', '6tMqYyCVU2aKgUahMvJz2S', '2020-07-23'), ('Trueno', 'Taiu', '5TomkIGGAC7JHVbqvPPUR5', '2020-07-23'), ('Trueno', 'Taiu', '03e3pnRddWmYOVQlzs3Qnb', '2020-07-23'), ('Trueno', 'Taiu', '66N8wrzpsNbOUQxHMzOUfv', '2020-07-23'), ('Trueno', 'WOS', '1fJjC8C0BdgUHlKIP2ODQf', '2020-07-23'), ('Trueno', 'Taiu', '1fJjC8C0BdgUHlKIP2ODQf', '2020-07-23'), ('Trueno', 'Taiu', '6Cgj3VVfcYJN9vZbMVcKRd', '2020-07-23'), ('Trueno', 'Taiu', '2Kv37yc67cNNAujkActU6t', '2020-07-23'), ('Trueno', 'Nicki Nicole', '0TUW9faHNaBmi89wsYGp9y', '2020-07-23'), ('Trueno', 'Bizarrap', '0TUW9faHNaBmi89wsYGp9y', '2020-07-23'), ('Trueno', 'Taiu', '0TUW9faHNaBmi89wsYGp9y', '2020-07-23'), ('Trueno', 'Tiago PZK', '3UytZp1xDkNkROyIwE7DbY', '2022-10-06'), ('Trueno', 'Bizarrap', '3monFIRNCDt9we37DDvCCK', '2022-02-17'), ('Trueno', 'Tiago PZK', '6P096SaOzrgGxs8NWOfX7M', '2021-10-21'), ('Trueno', 'Bizarrap', '0egDphqMcXZ9YeqvSBVRu3', '2020-10-29'), ('Trueno', 'ACRU', '0egDphqMcXZ9YeqvSBVRu3', '2020-10-29'), ('Trueno', 'Taiu', '3wDWtX5zoRf11qV08d9wAG', '2020-03-25'), ('Trueno', 'Taiu', '2KzTj36yIV9BvqiXRWqvyv', '2020-02-20'), ('Homer El Mero Mero', 'FMK', '3LMrjx3PnGc2Od9loPTC45', '2022-06-16'), ('Homer El Mero Mero', 'Tiago PZK', '3LMrjx3PnGc2Od9loPTC45', '2022-06-16'), ('Homer El Mero Mero', 'Bardero$', '0Kros3xlCtmyHTarar1SZ3', '2022-10-18'), ('Homer El Mero Mero', 'FMK', '3G2Dtbtw8YLZGpWVgidQei', '2022-06-15'), ('Homer El Mero Mero', 'Tiago PZK', '3G2Dtbtw8YLZGpWVgidQei', '2022-06-15'), ('Homer El Mero Mero', 'Franky Style', '1Eq3OV3WQDY71m3cryNTHE', '2022-05-20'), ('Homer El Mero Mero', 'Franky Style', '1qta1MH4tVSsOheGYCdaxZ', '2022-05-20'), ('Homer El Mero Mero', 'Kaleb Di Masi', '4JFdVCQnqWLl1wG7hVlOGx', '2022-01-14'), ('Homer El Mero Mero', 'Lil Troca', '6Bo4B8mxqcy9MWwnbpCqTr', '2021-10-15'), ('Homer El Mero Mero', 'Perro Primo', '539LAQobYbCmWNyEhy1ywq', '2021-07-16'), ('ACRU', 'Veeyam', '5O8xik2WSUwVua5X2AZwwY', '2021-10-26'), ('ACRU', 'Veeyam', '4Is4s9GapAryQjvWDxulED', '2021-10-26'), ('ACRU', 'Veeyam', '5Jf2HO3ev1mBpWRIuuDbit', '2021-10-26'), ('ACRU', 'Veeyam', '5N3M8SHDVp7UHArixHePYG', '2021-10-05'), ('ACRU', 'Veeyam', '7b3gQL4a0Zf7rErbM0xcgv', '2021-09-07'), ('ACRU', 'Veeyam', '6tZm9vnnt0nSZJIRRZFtan', '2021-08-17'), ('ACRU', 'Veeyam', '3rXGHdUlh8bvDMvkuI3qdB', '2021-05-12'), ('ACRU', 'Bizarrap', '0egDphqMcXZ9YeqvSBVRu3', '2020-10-29'), ('ACRU', 'Trueno', '0egDphqMcXZ9YeqvSBVRu3', '2020-10-29'), ('Franky Style', 'Homer El Mero Mero', '4w11vdxDMTxwfh0Yy6V8Pv', '2020-12-11'), ('Franky Style', 'Bardero$', '1remv0DkW7pPBHsOGtyON1', '2020-10-23'), ('Franky Style', 'Chulu', '1remv0DkW7pPBHsOGtyON1', '2020-10-23'), ('Franky Style', 'Bardero$', '3kJkC0W2QMs8acfS4ssB6f', '2020-10-23'), ('Franky Style', 'Chulu', '2UhrNOPfjhpqH4v0fMYLIQ', '2020-10-23'), ('Franky Style', 'Bardero$', '2UhrNOPfjhpqH4v0fMYLIQ', '2020-10-23'), ('Franky Style', 'Bardero$', '6keZVGv8A4cX5Q4NfIV61t', '2020-10-23'), ('Franky Style', 'Bardero$', '4ZPHivCYG5nqs7UWwXbgK2', '2020-10-23'), ('Franky Style', 'Homer El Mero Mero', '4ZPHivCYG5nqs7UWwXbgK2', '2020-10-23'), ('Franky Style', 'Bardero$', '5B4EkBBQlZJsS1qWPjmq0B', '2020-10-23'), ('Franky Style', 'Homer El Mero Mero', '5B4EkBBQlZJsS1qWPjmq0B', '2020-10-23'), ('Franky Style', 'Lil Troca', '5B4EkBBQlZJsS1qWPjmq0B', '2020-10-23'), ('Franky Style', 'Bardero$', '4ATtunTXSzWkqUTqOZjZts', '2020-10-23'), ('Franky Style', 'Lil Troca', '4ATtunTXSzWkqUTqOZjZts', '2020-10-23'), ('Franky Style', 'Bardero$', '7JyowjMQnogOnpVk0HCKfG', '2020-10-23'), ('Franky Style', 'Bardero$', '2e11sHNDJS7cYDLgOiVz6X', '2020-10-23'), ('Franky Style', 'Chulu', '2e11sHNDJS7cYDLgOiVz6X', '2020-10-23'), ('Franky Style', 'Bardero$', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Franky Style', 'Homer El Mero Mero', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Franky Style', 'Lil Troca', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Franky Style', 'Chulu', '50WsYaCHkz2XlACA2CEg6o', '2020-10-23'), ('Franky Style', 'Homer El Mero Mero', '1Eq3OV3WQDY71m3cryNTHE', '2022-05-20'), ('Franky Style', 'Homer El Mero Mero', '1qta1MH4tVSsOheGYCdaxZ', '2022-05-20'), ('Franky Style', 'Obie Wanshot', '3njm1G9QgV5Lis9Iu4KNiL', '2021-08-04'), ('Kris Alaniz', 'Ramiro Jota', '2yoiUhn9PS8tT1uWTbYhw6', '2020-06-19'), ('Kris Alaniz', 'Ramiro Jota', '54BAh1myN0H1h3biO1vgIv', '2020-06-19'), ('Kris Alaniz', 'Ramiro Jota', '2g87eqWEIBfXp0nTXV0Ndi', '2020-06-19'), ('Kris Alaniz', 'XXL Irione', '0RzvjxUKpTC0nebvwUoGmd', '2020-05-30'), ('Kris Alaniz', 'Alika', '7txvBb6pgzeF2q4aLpKvza', '2021-07-02'), ('Kris Alaniz', 'Lauphan', '4Kty7fCDSpuuqCvlKMrYIO', '2019-11-13'), ('Kris Alaniz', 'Fede Flores', '5eP6tvfjz5MBygNwfUgXBD', '2019-11-13'), ('Kris Alaniz', 'Ramiro Jota', '4l4nCJCHwavgnq63aSxe6z', '2019-11-13'), ('Kris Alaniz', 'SidiRum', '6zptPkctpCK5t7zC9xJFm9', '2019-11-13'), ('Fianru', 'Homer El Mero Mero', '555TZaGCOxevdgRb9uuEyD', '2018-09-21'), ('Fianru', 'Marcianos Crew', '2cCBGtvPTQ365dyXU1efQp', '2018-09-21'), ('Fianru', 'Cazzu', '7k6ZBVP70i5SrnqIoOhkqC', '2018-09-21'), ('Fianru', 'Mike Southside', '7wDqf8VIXrJs21mnijNO42', '2022-10-06'), ('Lil Troca', 'Lucho SSJ', '7h2s8YoaToz6Fvimw7ufnf', '2021-12-10'), ('Lil Troca', 'Homer El Mero Mero', '6Bo4B8mxqcy9MWwnbpCqTr', '2021-10-15'), ('Lil Troca', 'Lucho SSJ', '1yxmzDLltONJYU9iXFYM6D', '2021-05-04'), ('Lil Troca', 'Homer El Mero Mero', '6bp3wTx0LWSWEjxmLERBZo', '2021-03-16'), ('Lil Troca', 'NEGRO DUB', '6bp3wTx0LWSWEjxmLERBZo', '2021-03-16'), ('Lil Troca', 'Homer El Mero Mero', '7gVHTVG1SaqJMiKfmLgNKw', '2020-12-16'), ('Veeyam', 'ACRU', '5IW0NUt3mkyPdy46TQy8qQ', '2022-05-19'), ('Veeyam', 'ACRU', '5O8xik2WSUwVua5X2AZwwY', '2021-10-26'), ('Veeyam', 'ACRU', '4Is4s9GapAryQjvWDxulED', '2021-10-26'), ('Veeyam', 'ACRU', '5Jf2HO3ev1mBpWRIuuDbit', '2021-10-26'), ('Veeyam', 'ACRU', '5N3M8SHDVp7UHArixHePYG', '2021-10-05'), ('Veeyam', 'ACRU', '7b3gQL4a0Zf7rErbM0xcgv', '2021-09-07'), ('Veeyam', 'ACRU', '6tZm9vnnt0nSZJIRRZFtan', '2021-08-17'), ('Veeyam', 'ACRU', '3rXGHdUlh8bvDMvkuI3qdB', '2021-05-12'), ('Marcianos Crew', 'Obie Wanshot', '3YfXZo3b4sYDkcy9UlCJmr', '2017-11-08'), ('Marcianos Crew', 'Fianru', '233fLBooQLNPg9bske7y3z', '2017-11-08'), ('Marcianos Crew', 'Fianru', '67OASBDkkauUCqciUrRdBw', '2017-11-08'), ('Marcianos Crew', 'Fianru', '0kaXhNzS8ipCc2K0YQiRyd', '2017-11-08'), ('Marcianos Crew', 'Miss Bolivia', '6uY9ol9Xi6zcWTzjk35boE', '2012'), ('Marcianos Crew', 'Fidel Nadal', '7tAN98StXAQBIkKseK4cfT', '2008'), ('Marcianos Crew', 'Homer El Mero Mero', '11SlUn8ffDBqp1u0moI8y2', '2020-10-23'), ('Marcianos Crew', 'YSY A', '29Nzo1DUgMGtxU6op3ZI9n', '2019-04-22'), ('Marcianos Crew', 'Homer El Mero Mero', '201q0mutyf5H2yVl9fISbB', '2019-02-22'), ('Marcianos Crew', 'Homer El Mero Mero', '44bt91U70qCiBC8EF34YLY', '2019-01-18'), ('Marcianos Crew', 'Homer El Mero Mero', '1Uoa1hfThZUSvoKTEXsHcm', '2018-10-22'), ('Marcianos Crew', 'Mike Southside', '5TxuYK3OVRIBIUqujVa6tR', '2018-09-07'), ('Marcianos Crew', 'Bardero$', '34b0FvG8CvJxyK7TmTNpad', '2018-02-08'), ('Marcianos Crew', 'Obie Wanshot', '001dIBkGfnvIajX9B0699L', '2017-11-13'), ('Marcianos Crew', 'KHEA', '6HvD8JhIe9Mq6gyql32rYb', '2019-11-01'), ('Marcianos Crew', 'KHEA', '51dqvo5QYezL3rjL6qjksA', '2019-11-01'), ('Marcianos Crew', 'YSY A', '74MujHggasHq5FMkTKsNkA', '2019-11-01'), ('Marcianos Crew', 'Lucho SSJ', '2lllz0GoFtRdHMaqGlqcoL', '2019-11-01'), ('KHEA', 'Asan', '4ysuHVUGF4kMlpn3naUuXT', '2020-05-15'), ('KHEA', 'Rusherking', '3AiiL8Idhbmm3vDf2dmyRW', '2022-08-11'), ('KHEA', 'FMK', '3AiiL8Idhbmm3vDf2dmyRW', '2022-08-11'), ('KHEA', 'Rusherking', '4kiTIRY4o3o9on5oYV584L', '2022-03-03'), ('KHEA', 'FMK', '2VZmXemWIKoKxJjbA2WHJE', '2022-01-24'), ('KHEA', 'Maria Becerra', '79uVK69HCjcVEKsVcl4DAB', '2021-08-11'), ('KHEA', 'Maria Becerra', '74UFJflGMdA46TltO7XPSB', '2021-05-13'), ('Frijo', 'Punga', '3R7QUoJOF3HBekCl0qsCzw', '2022-09-09'), ('FMK', 'Dani Ribba', '0RHovjWjcv1FhEGaKlrqRw', '2022-03-19'), ('FMK', 'Tiago PZK', '1cqEej7ZM0hmpjAmMubjiU', '2022-03-19'), ('FMK', 'Rusherking', '1cqEej7ZM0hmpjAmMubjiU', '2022-03-19'), ('FMK', 'Maria Becerra', '1x8e8iyMYuaZ92j1hfTCcW', '2022-03-19'), ('FMK', 'Tiago PZK', '5xLa7ej9QQSytCOfOFRMbs', '2022-03-19'), ('FMK', 'Dani Ribba', '1yihMXFt0zsG86nue7u41U', '2022-03-10'), ('FMK', 'Tiago PZK', '647WGadTATihqNaGBz8duA', '2022-03-10'), ('FMK', 'Rusherking', '647WGadTATihqNaGBz8duA', '2022-03-10'), ('FMK', 'Maria Becerra', '1v5QhGtlv2bkpUz2kAXTCq', '2022-03-10'), ('FMK', 'Tiago PZK', '37hA11IuYK6W5SAfiVNvfa', '2022-03-10'), ('FMK', 'Homer El Mero Mero', '3G2Dtbtw8YLZGpWVgidQei', '2022-06-15'), ('FMK', 'Tiago PZK', '3G2Dtbtw8YLZGpWVgidQei', '2022-06-15'), ('FMK', 'Rusherking', '5aQtcr0PfEtk4FZCz5rLdV', '2022-05-26'), ('FMK', 'Dani Ribba', '5JH2j55T7jhjhr4ZsC8tZs', '2022-03-10'), ('FMK', 'KHEA', '2VZmXemWIKoKxJjbA2WHJE', '2022-01-24'), ('FMK', 'Maria Becerra', '5NMPnWUDvXr8Vnauv7I7YZ', '2022-01-13'), ('FMK', 'El Retutu', '0KvtuCa8aGD8SR39aLLLj5', '2021-12-15'), ('FMK', 'Tiago PZK', '5A1vGSsqw0UQCBxHOyPOjG', '2021-11-08'), ('Lucho SSJ', 'Lil Troca', '1DQl9DmNaXosM28N4yenl9', '2022-06-15'), ('Lucho SSJ', 'Obie Wanshot', '5zd10IQXHEXq6qVoLNJeqD', '2021-03-29'), ('Lucho SSJ', 'Lil Troca', '4eEzqkj3Rvu9iiBhCFYgoE', '2021-03-29'), ('Lucho SSJ', 'Lil Troca', '09i6xDeW1dr75QXna7WbK9', '2020-04-13'), ('Lucho SSJ', 'KHEA', '0wb9hhozBdZ1GNcxp2UFJ3', '2020-04-13'), ('Lucho SSJ', 'Lil Troca', '6YkcgRD9XQEycwA0sTogpL', '2022-09-08'), ('Lucho SSJ', 'Lil Troca', '7h2s8YoaToz6Fvimw7ufnf', '2021-12-10'), ('Lucho SSJ', 'Lil Troca', '1yxmzDLltONJYU9iXFYM6D', '2021-05-04'), ('Pekeño 77', 'Cazzu', '2TGZPm8tRKgepIZ6RuE1tt', '2020-09-04'), ('Pekeño 77', 'Homer El Mero Mero', '2TGZPm8tRKgepIZ6RuE1tt', '2020-09-04'), ('Pekeño 77', 'Homer El Mero Mero', '0de0MI3ZOuXxmGVeTnszIF', '2020-09-04'), ('Pekeño 77', 'Jairo Vera', '0P7k4vtVPZxMuWR7wYTHHw', '2022-07-06'), ('Pekeño 77', 'Mario La Moña', '2BwUX9xv05GfC57oquKdfv', '2021-11-29')]


In [7]:
filename = f"red_artistas_argentinos_nico.npz"
# np.savez(filename, cuadrivectores = cuadrivectores_artistas_argentinos)
np.savez(filename, cuadrivectores = cuadrivectores_artistas_argentinos)

In [ ]:
red_artistas= nx.Graph()
enlaces = []
for i in cuadrivectores_artistas_argentinos:
    enlaces.append(i[0:2])
print(enlaces)
red_artistas.add_edges_from(enlaces)
nx.write_gexf(red_artistas, "artistas_argentinos_ari.gexf")


In [ ]:
#guardo
import pickle

# save graph object to file
pickle.dump(red_artistas, open('artistas_argentinos_ari', 'wb'))

Importo la red

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True) # Montamos nuestra unidad de Google Drive

Mounted at /content/drive


In [ ]:
filename = '/content/drive/My Drive/Redes Complejas/TP Final/Archivos/red_rosalia_hasta_64.npz'
npzfile = np.load(filename)